In [21]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [3]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential

In [4]:
import numpy as np
from glob import glob

In [5]:
#resize all images
IMAGE_SIZE = [224,224]
train_path = 'train'
test_path = 'test'

In [6]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

In [7]:
#dont train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [8]:
train_folders = glob('train/*')
train_folders

['train\\diseased cotton leaf',
 'train\\diseased cotton plant',
 'train\\fresh cotton leaf',
 'train\\fresh cotton plant']

In [9]:
test_folders = glob('test/*')
test_folders

['test\\diseased cotton leaf',
 'test\\diseased cotton plant',
 'test\\fresh cotton leaf',
 'test\\fresh cotton plant']

In [10]:
#flattern layer
x = Flatten()(vgg16.output)

In [11]:
prediction = Dense(4,activation='softmax')(x)
#create model
model = Model(inputs=vgg16.input,outputs=prediction)

In [12]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)

In [15]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [16]:
training_set = train_datagen.flow_from_directory('train',
                                                 target_size=(224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [17]:
test_set = test_datagen.flow_from_directory('test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 106 images belonging to 4 classes.


In [18]:
#fit the model
r = model.fit_generator(training_set,
                        validation_data = test_set,
                        epochs=20,
                        steps_per_epoch = len(training_set),
                        validation_steps = len(test_set))

Epoch 1/20
61/61 [==============================] - 71s 1s/step - loss: 0.6460 - accuracy: 0.7581 - val_loss: 0.5607 - val_accuracy: 0.8679
Epoch 2/20
61/61 [==============================] - 25s 414ms/step - loss: 0.2336 - accuracy: 0.9108 - val_loss: 0.1659 - val_accuracy: 0.9057
Epoch 3/20
61/61 [==============================] - 25s 403ms/step - loss: 0.1981 - accuracy: 0.9257 - val_loss: 0.2411 - val_accuracy: 0.9434
Epoch 4/20
61/61 [==============================] - 20s 324ms/step - loss: 0.1335 - accuracy: 0.9493 - val_loss: 0.0641 - val_accuracy: 0.9528
Epoch 5/20
61/61 [==============================] - 20s 322ms/step - loss: 0.1058 - accuracy: 0.9713 - val_loss: 0.0574 - val_accuracy: 0.9340
Epoch 6/20
61/61 [==============================] - 20s 320ms/step - loss: 0.1049 - accuracy: 0.9672 - val_loss: 0.0113 - val_accuracy: 0.9434
Epoch 7/20
61/61 [==============================] - 20s 323ms/step - loss: 0.0781 - accuracy: 0.9754 - val_loss: 0.1523 - val_accuracy: 0.9340
Ep

In [19]:
from keras.models import load_model
model.save('cotton_disease_vgg16.h5')